In [35]:
import pandas as pd
import config_prod as config
from datetime import date, timedelta
import phood_api
import numpy as np

In [36]:
names = pd.read_excel('Phood Solutions List Names.xlsx')
names=names.drop([0,1,2])
names=names.iloc[:,0:2]

names.columns = ['PLU', 'Item_Names']
names.reset_index(drop=True)
ndic = names.set_index('PLU').to_dict('index')

In [37]:
locations = ['dedham', 'hingham', 'westford', 'westhartford', 'cranston']
unnamedplu = []
masterlist=[]
#print(ndic)
locationinfo = {51: {'name': 'Dedham'}, 52: {'name': 'Hingham'}, 53: {'name': 'Westford'}, 54: {'name': 'West hartford'}, 55: {'name': 'Cranston'}}
for location in locations:
    phoodServer = phood_api.PhoodAPI(config.base_url)
    username = location
    res = phoodServer.login(username, config.password).json()
    endDate = date.today() #- timedelta(days=date.today().weekday())
    startDate = date(2019,10, 1)
    #endDate = date.today()- timedelta(days=1)
    #startDate = endDate - timedelta(days=6)
    res = phoodServer.get_food_logs(startDate, endDate).json()
    PLUEntries = filter(lambda x: x['clientId'], res)
    orderedEntries = list(sorted(PLUEntries, key=lambda x: x["loggedTime"]))
    hotBar,saladBar,soupBar = set(),set(),set()
    res = phoodServer.getPersistentItems().json()
    for pItem in res:
        if pItem['station'].lower().startswith('hot bar'):
            hotBar.add(pItem['clientId'])
        elif pItem['station'].lower().startswith('salad bar'):
            saladBar.add(pItem['clientId'])
        elif pItem['station'].lower().startswith('soup'):
            soupBar.add(pItem['clientId'])
        else:
            print(f'unclassified item {pItem}')

    res = phoodServer.getPanInfo().json()
    panInfo = {}
    for pan in res:
        panInfo[pan["id"]] = {"name": pan["name"], "weight": pan["weightQuantity"]}
    #print(panInfo)
            
    for item in orderedEntries:
        item['clientId'] = item['clientId'].replace("-", "")
        #print(item['clientId'])
        if item['itemName'] in ndic.keys():
            item['itemName'] = ndic[int(item['clientId'])]['Item_Names']
        else:
            unnamedplu.append(item)
        #print(item['Item_Names'])
        if item['clientId'] in hotBar:
            item['station'] = 'HB'
        elif item['clientId'] in saladBar:
            item['station'] = 'SB'
        elif item['clientId'] in soupBar:
            item['station'] = 'SO'
        item['Location'] = locationinfo[item['locationId']]['name']
        item['Served']=item['Saved']=item['Sold']=item['Shrink']=0
        if 'panId' in item and item['panId']:
            panWeight = panInfo[item['panId']]['weight']
        else:
            panWeight = item['panWeight'] 
        if item['actionTaken']=='Served':
            item['Served'] = +item['quantity'] - panWeight
        if item['actionTaken']=='Discarded':
            item['Shrink'] = +item['quantity'] - panWeight
        if item['actionTaken']=='Saved':
            item['Saved'] = +item['quantity'] - panWeight
        item['Sold'] = item['Served'] - item['Saved'] - item['Shrink']
        if item['clientId'] == '00449791900':
            item['station'] = 'HB'
            #print(item)
        
    masterlist.extend(orderedEntries)

In [38]:
raw = pd.DataFrame(masterlist) 

In [39]:
rawfile=raw

In [40]:
raw.columns = ['Date_Produced', 'Log_ID', 'Item_Name', 'Item_Type', 'PLU', 'Logged_Time', 'Meal', 'Action_Taken', 'Action_Reason', 'Quantity', 'Pan_ID', 'Pan_Weight', 'Unit', 'Station', 'Employee_Name', 'Location_ID','Location', 'Served','Saved','Sold','Shrink']


In [41]:
rawfile = rawfile.rename(columns={"Logged_Time": "timestamped"})
rawfile['timestamped']= pd.to_datetime(rawfile['timestamped'], utc = True) 

In [42]:
df = rawfile[rawfile['timestamped']< '2019-11-1']
df

,Date_Produced,Log_ID,Item_Name,Item_Type,PLU,timestamped,Meal,Action_Taken,Action_Reason,Quantity,...,Pan_Weight,Unit,Station,Employee_Name,Location_ID,Location,Served,Saved,Sold,Shrink
0,2019-09-30,43120,Spring Mix,Menu Item,46000063050,2019-10-01 06:18:52+00:00,Breakfast,Served,Start,6.45954,...,1.94,Lbs,SB,None,51,Dedham,4.51954,0.0,4.51954,0.00000
1,2019-09-30,43122,Romaine Lettuce,Menu Item,46000055863,2019-10-01 06:19:06+00:00,Breakfast,Served,Start,6.45954,...,1.94,Lbs,SB,None,51,Dedham,4.51954,0.0,4.51954,0.00000
2,2019-09-30,43123,Romaine Lettuce,Menu Item,46000055863,2019-10-01 06:19:20+00:00,Breakfast,Served,Start,6.34931,...,1.94,Lbs,SB,None,51,Dedham,4.40931,0.0,4.40931,0.00000
3,2019-09-30,43124,Baby Spinach,Menu Item,46000055923,2019-10-01 06:19:35+00:00,Breakfast,Served,Start,6.76819,...,1.94,Lbs,SB,None,51,Dedham,4.82819,0.0,4.82819,0.00000
4,2019-09-30,43194,Plantains,Menu Item,46000001661,2019-10-01 07:06:17+00:00,Breakfast,Served,Start,6.74615,...,1.94,Lbs,HB,None,51,Dedham,4.80615,0.0,4.80615,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37600,2019-10-31,69482,Collard Greens & Tomatoes,Menu Item,46000030030,2019-10-31 21:15:58+00:00,Breakfast,Discarded,Over Produced,5.00449,...,0.95,Lbs,HB,None,55,Cranston,0.00000,0.0,-4.05449,4.05449
37601,2019-10-31,69484,Chicken Tenders,Menu Item,46000007582,2019-10-31 21:16:22+00:00,Breakfast,Discarded,Over Produced,1.76370,...,0.95,Lbs,HB,None,55,Cranston,0.00000,0.0,-0.81370,0.81370
37602,2019-10-31,69485,Waffle Fries,Menu Item,46000064878,2019-10-31 21:16:44+00:00,Breakfast,Discarded,Over Produced,2.55736,...,1.90,Lbs,HB,None,55,Cranston,0.00000,0.0,-0.65736,0.65736
37603,2019-10-31,69486,Potato Samosa,Menu Item,46000057583,2019-10-31 21:17:04+00:00,Breakfast,Discarded,Over Produced,5.46746,...,1.90,Lbs,HB,None,55,Cranston,0.00000,0.0,-3.56746,3.56746


In [43]:
images = pd.read_csv('rek-test-images-all.csv')

In [44]:
images

,timestamp,locationId,imageId,imageKey
0,2019-09-16T17:01:13.585000,51,96fb9610-d8a3-11e9-a35a-0de7ce4e8d91,51/96fb9610d8a311e9a35a0de7ce4e8d91
1,2019-09-16T17:15:20.774000,51,8ff25e60-d8a5-11e9-a35a-0de7ce4e8d91,51/8ff25e60d8a511e9a35a0de7ce4e8d91
2,2019-09-16T17:15:36.551000,51,9959bf70-d8a5-11e9-a35a-0de7ce4e8d91,51/9959bf70d8a511e9a35a0de7ce4e8d91
3,2019-09-16T17:15:52.755000,51,a3024830-d8a5-11e9-ae9b-73759b4dfb23,51/a3024830d8a511e9ae9b73759b4dfb23
4,2019-09-16T17:15:54.492000,51,a40b53c0-d8a5-11e9-ae9b-73759b4dfb23,51/a40b53c0d8a511e9ae9b73759b4dfb23
...,...,...,...,...
33757,2019-11-05T23:56:30.501000,54,e346bd50-0027-11ea-90c3-45c3f76c0e47,54/e346bd50002711ea90c345c3f76c0e47
33758,2019-11-06T00:06:43.342000,54,508edae0-0029-11ea-8246-012e5e367d18,54/508edae0002911ea8246012e5e367d18
33759,2019-11-06T00:20:08.856000,55,30ae8980-002b-11ea-8246-012e5e367d18,55/30ae8980002b11ea8246012e5e367d18
33760,2019-11-06T00:22:27.407000,54,8343bdf0-002b-11ea-90c3-45c3f76c0e47,54/8343bdf0002b11ea90c345c3f76c0e47


In [45]:
images = images.rename(columns={"timestamp": "timestamped"})
images['timestamped']= pd.to_datetime(images['timestamped'], utc = True) 


In [46]:
imagesnew = images[images['timestamped'] >= '2019-10-1']
imagesnew = imagesnew[imagesnew['timestamped'] < '2019-11-1']
imagesnew

,timestamped,locationId,imageId,imageKey
4035,2019-10-01 00:06:49.553000+00:00,52,5d639410-e3df-11e9-8246-012e5e367d18,52/5d639410e3df11e98246012e5e367d18
4036,2019-10-01 00:07:02.519000+00:00,52,651e0870-e3df-11e9-90c3-45c3f76c0e47,52/651e0870e3df11e990c345c3f76c0e47
4037,2019-10-01 00:14:47.436000+00:00,53,7a3abcc0-e3e0-11e9-90c3-45c3f76c0e47,53/7a3abcc0e3e011e990c345c3f76c0e47
4038,2019-10-01 00:18:59.163000+00:00,52,10453ab0-e3e1-11e9-90c3-45c3f76c0e47,52/10453ab0e3e111e990c345c3f76c0e47
4039,2019-10-01 00:20:55.193000+00:00,55,556e0090-e3e1-11e9-8246-012e5e367d18,55/556e0090e3e111e98246012e5e367d18
...,...,...,...,...
31468,2019-10-31 22:35:02.223000+00:00,53,ad91f5f0-fc2e-11e9-8246-012e5e367d18,53/ad91f5f0fc2e11e98246012e5e367d18
31469,2019-10-31 22:35:23.892000+00:00,53,ba7c6340-fc2e-11e9-8246-012e5e367d18,53/ba7c6340fc2e11e98246012e5e367d18
31470,2019-10-31 22:35:41.625000+00:00,53,c50e3a90-fc2e-11e9-90c3-45c3f76c0e47,53/c50e3a90fc2e11e990c345c3f76c0e47
31471,2019-10-31 22:40:20.308000+00:00,55,6b29e140-fc2f-11e9-90c3-45c3f76c0e47,55/6b29e140fc2f11e990c345c3f76c0e47


In [47]:
imagesnew['timestamped'] = pd.to_datetime(imagesnew['timestamped']) - timedelta(hours=4) - timedelta(seconds=48)

In [48]:
imagesnew

,timestamped,locationId,imageId,imageKey
4035,2019-09-30 20:06:01.553000+00:00,52,5d639410-e3df-11e9-8246-012e5e367d18,52/5d639410e3df11e98246012e5e367d18
4036,2019-09-30 20:06:14.519000+00:00,52,651e0870-e3df-11e9-90c3-45c3f76c0e47,52/651e0870e3df11e990c345c3f76c0e47
4037,2019-09-30 20:13:59.436000+00:00,53,7a3abcc0-e3e0-11e9-90c3-45c3f76c0e47,53/7a3abcc0e3e011e990c345c3f76c0e47
4038,2019-09-30 20:18:11.163000+00:00,52,10453ab0-e3e1-11e9-90c3-45c3f76c0e47,52/10453ab0e3e111e990c345c3f76c0e47
4039,2019-09-30 20:20:07.193000+00:00,55,556e0090-e3e1-11e9-8246-012e5e367d18,55/556e0090e3e111e98246012e5e367d18
...,...,...,...,...
31468,2019-10-31 18:34:14.223000+00:00,53,ad91f5f0-fc2e-11e9-8246-012e5e367d18,53/ad91f5f0fc2e11e98246012e5e367d18
31469,2019-10-31 18:34:35.892000+00:00,53,ba7c6340-fc2e-11e9-8246-012e5e367d18,53/ba7c6340fc2e11e98246012e5e367d18
31470,2019-10-31 18:34:53.625000+00:00,53,c50e3a90-fc2e-11e9-90c3-45c3f76c0e47,53/c50e3a90fc2e11e990c345c3f76c0e47
31471,2019-10-31 18:39:32.308000+00:00,55,6b29e140-fc2f-11e9-90c3-45c3f76c0e47,55/6b29e140fc2f11e990c345c3f76c0e47


In [49]:
imagesnew['year'] = imagesnew['timestamped'].dt.year
imagesnew['month'] = imagesnew['timestamped'].dt.month
imagesnew['day'] = imagesnew['timestamped'].dt.day
imagesnew['hour'] = imagesnew['timestamped'].dt.hour
imagesnew['minute'] = imagesnew['timestamped'].dt.minute
imagesnew['second'] = imagesnew['timestamped'].dt.second
#imagesnew.loc[(imagesnew.second <= 20) & (imagesnew.second > 0), "second"] = 20
#imagesnew.loc[(imagesnew.second <= 40) & (imagesnew.second > 20), "second"] = 40
#imagesnew.loc[(imagesnew.second <= 59) & (imagesnew.second > 40), "second"] = 59
#imagesnew['second'] = np.where(imagesnew['second']<=30, 30, 59)
#imagesnew = imagesnew.drop(['second'], 1)

In [50]:
imagesnew

,timestamped,locationId,imageId,imageKey,year,month,day,hour,minute,second
4035,2019-09-30 20:06:01.553000+00:00,52,5d639410-e3df-11e9-8246-012e5e367d18,52/5d639410e3df11e98246012e5e367d18,2019,9,30,20,6,1
4036,2019-09-30 20:06:14.519000+00:00,52,651e0870-e3df-11e9-90c3-45c3f76c0e47,52/651e0870e3df11e990c345c3f76c0e47,2019,9,30,20,6,14
4037,2019-09-30 20:13:59.436000+00:00,53,7a3abcc0-e3e0-11e9-90c3-45c3f76c0e47,53/7a3abcc0e3e011e990c345c3f76c0e47,2019,9,30,20,13,59
4038,2019-09-30 20:18:11.163000+00:00,52,10453ab0-e3e1-11e9-90c3-45c3f76c0e47,52/10453ab0e3e111e990c345c3f76c0e47,2019,9,30,20,18,11
4039,2019-09-30 20:20:07.193000+00:00,55,556e0090-e3e1-11e9-8246-012e5e367d18,55/556e0090e3e111e98246012e5e367d18,2019,9,30,20,20,7
...,...,...,...,...,...,...,...,...,...,...
31468,2019-10-31 18:34:14.223000+00:00,53,ad91f5f0-fc2e-11e9-8246-012e5e367d18,53/ad91f5f0fc2e11e98246012e5e367d18,2019,10,31,18,34,14
31469,2019-10-31 18:34:35.892000+00:00,53,ba7c6340-fc2e-11e9-8246-012e5e367d18,53/ba7c6340fc2e11e98246012e5e367d18,2019,10,31,18,34,35
31470,2019-10-31 18:34:53.625000+00:00,53,c50e3a90-fc2e-11e9-90c3-45c3f76c0e47,53/c50e3a90fc2e11e990c345c3f76c0e47,2019,10,31,18,34,53
31471,2019-10-31 18:39:32.308000+00:00,55,6b29e140-fc2f-11e9-90c3-45c3f76c0e47,55/6b29e140fc2f11e990c345c3f76c0e47,2019,10,31,18,39,32


In [51]:
imagesnew['logtime'] = pd.to_datetime(imagesnew[['year', 'month', 'day', 'hour', 'minute','second']])
imagesnew = imagesnew.drop(['year', 'month', 'day', 'hour', 'minute','second','timestamped'], 1)

In [52]:
imagesnew#.to_csv('join1.csv')
imagesnew

,locationId,imageId,imageKey,logtime
4035,52,5d639410-e3df-11e9-8246-012e5e367d18,52/5d639410e3df11e98246012e5e367d18,2019-09-30 20:06:01
4036,52,651e0870-e3df-11e9-90c3-45c3f76c0e47,52/651e0870e3df11e990c345c3f76c0e47,2019-09-30 20:06:14
4037,53,7a3abcc0-e3e0-11e9-90c3-45c3f76c0e47,53/7a3abcc0e3e011e990c345c3f76c0e47,2019-09-30 20:13:59
4038,52,10453ab0-e3e1-11e9-90c3-45c3f76c0e47,52/10453ab0e3e111e990c345c3f76c0e47,2019-09-30 20:18:11
4039,55,556e0090-e3e1-11e9-8246-012e5e367d18,55/556e0090e3e111e98246012e5e367d18,2019-09-30 20:20:07
...,...,...,...,...
31468,53,ad91f5f0-fc2e-11e9-8246-012e5e367d18,53/ad91f5f0fc2e11e98246012e5e367d18,2019-10-31 18:34:14
31469,53,ba7c6340-fc2e-11e9-8246-012e5e367d18,53/ba7c6340fc2e11e98246012e5e367d18,2019-10-31 18:34:35
31470,53,c50e3a90-fc2e-11e9-90c3-45c3f76c0e47,53/c50e3a90fc2e11e990c345c3f76c0e47,2019-10-31 18:34:53
31471,55,6b29e140-fc2f-11e9-90c3-45c3f76c0e47,55/6b29e140fc2f11e990c345c3f76c0e47,2019-10-31 18:39:32


In [53]:
df['year'] = df['timestamped'].dt.year
df['month'] = df['timestamped'].dt.month
df['day'] = df['timestamped'].dt.day
df['hour'] = df['timestamped'].dt.hour
df['minute'] = df['timestamped'].dt.minute
df['second'] = df['timestamped'].dt.second
#df.loc[(df.second <= 20) & (df.second > 0), "second"] = 20
#df.loc[(df.second <= 40) & (df.second > 20), "second"] = 40
#df.loc[(df.second <= 59) & (df.second > 40), "second"] = 59
#df['second'] = np.where(df['second']<=30, 30, 59)
#df = df.drop(['second'], 1)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy o

In [54]:
df['logtime'] = pd.to_datetime(df[['year', 'month', 'day', 'hour', 'minute','second']])
df = df.drop(['year', 'month', 'day', 'hour', 'minute','second','timestamped'], 1)
df = df.sort_values(by='logtime')

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [55]:
df#.to_csv('join2.csv')
df

,Date_Produced,Log_ID,Item_Name,Item_Type,PLU,Meal,Action_Taken,Action_Reason,Quantity,Pan_ID,...,Unit,Station,Employee_Name,Location_ID,Location,Served,Saved,Sold,Shrink,logtime
15239,2019-10-01,43100,Organic Rainbow Cherry Tomato,Menu Item,46000016104,Breakfast,Served,Start,3.88014,175,...,Lbs,SB,None,53,Westford,2.03014,0.00000,2.03014,0.0,2019-10-01 05:18:13
15240,2019-10-01,43101,Organic Red Onions,Menu Item,46000055881,Breakfast,Served,Start,2.93215,175,...,Lbs,SB,None,53,Westford,1.08215,0.00000,1.08215,0.0,2019-10-01 05:19:07
15241,2019-10-01,43102,Red Cabbage,Menu Item,46000055765,Breakfast,Served,Start,2.46918,175,...,Lbs,SB,None,53,Westford,0.61918,0.00000,0.61918,0.0,2019-10-01 05:34:51
15242,2019-10-01,43103,Paleo Friendly Roasted Red Bell Peppers,Menu Item,46000028993,Breakfast,Served,Start,2.91010,175,...,Lbs,SB,None,53,Westford,1.06010,0.00000,1.06010,0.0,2019-10-01 05:35:15
15243,2019-10-01,43104,Cooked Red Beets,Menu Item,46000007337,Breakfast,Served,Start,4.05651,175,...,Lbs,SB,None,53,Westford,2.20651,0.00000,2.20651,0.0,2019-10-01 05:36:35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4897,2019-10-31,69514,Spring Mix,Menu Item,46000063050,Breakfast,Saved,End,3.74786,110,...,Lbs,SB,None,51,Dedham,0.00000,3.15786,-3.15786,0.0,2019-10-31 21:54:26
4898,2019-10-31,69515,Baby Spinach,Menu Item,46000055923,Breakfast,Saved,End,4.03446,110,...,Lbs,SB,None,51,Dedham,0.00000,3.44446,-3.44446,0.0,2019-10-31 21:54:48
4899,2019-10-31,69516,Arugula,Menu Item,46000055714,Breakfast,Saved,End,1.65347,111,...,Lbs,SB,None,51,Dedham,0.00000,1.30347,-1.30347,0.0,2019-10-31 21:56:35
4900,2019-10-31,69517,Romaine Lettuce,Menu Item,46000055863,Breakfast,Saved,End,5.51156,110,...,Lbs,SB,None,51,Dedham,0.00000,4.92156,-4.92156,0.0,2019-10-31 21:56:55


In [56]:
df53 = df[df.Location_ID==53].reset_index(drop=True)
df53#.to_csv('join1.csv')

,Date_Produced,Log_ID,Item_Name,Item_Type,PLU,Meal,Action_Taken,Action_Reason,Quantity,Pan_ID,...,Unit,Station,Employee_Name,Location_ID,Location,Served,Saved,Sold,Shrink,logtime
0,2019-10-01,43100,Organic Rainbow Cherry Tomato,Menu Item,46000016104,Breakfast,Served,Start,3.88014,175,...,Lbs,SB,None,53,Westford,2.03014,0.0,2.03014,0.00000,2019-10-01 05:18:13
1,2019-10-01,43101,Organic Red Onions,Menu Item,46000055881,Breakfast,Served,Start,2.93215,175,...,Lbs,SB,None,53,Westford,1.08215,0.0,1.08215,0.00000,2019-10-01 05:19:07
2,2019-10-01,43102,Red Cabbage,Menu Item,46000055765,Breakfast,Served,Start,2.46918,175,...,Lbs,SB,None,53,Westford,0.61918,0.0,0.61918,0.00000,2019-10-01 05:34:51
3,2019-10-01,43103,Paleo Friendly Roasted Red Bell Peppers,Menu Item,46000028993,Breakfast,Served,Start,2.91010,175,...,Lbs,SB,None,53,Westford,1.06010,0.0,1.06010,0.00000,2019-10-01 05:35:15
4,2019-10-01,43104,Cooked Red Beets,Menu Item,46000007337,Breakfast,Served,Start,4.05651,175,...,Lbs,SB,None,53,Westford,2.20651,0.0,2.20651,0.00000,2019-10-01 05:36:35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7016,2019-10-31,69425,Pumpkin Tortellini,Menu Item,46000029503,Breakfast,Discarded,Over Produced,4.27697,173,...,Lbs,HB,None,53,Westford,0.00000,0.0,-2.73697,2.73697,2019-10-31 20:31:39
7017,2019-10-31,69427,Chicken Cacciatore,Menu Item,46000005678,Breakfast,Discarded,Over Produced,6.08476,173,...,Lbs,HB,None,53,Westford,0.00000,0.0,-4.54476,4.54476,2019-10-31 20:32:04
7018,2019-10-31,69428,Garlic Parm Pork Loin with Broccoli Rabe,Menu Item,46000029505,Breakfast,Discarded,Over Produced,2.29281,173,...,Lbs,HB,None,53,Westford,0.00000,0.0,-0.75281,0.75281,2019-10-31 20:32:21
7019,2019-10-31,69430,Macaroni & Cheese,Menu Item,46000060414,Breakfast,Discarded,Over Produced,8.13506,173,...,Lbs,HB,None,53,Westford,0.00000,0.0,-6.59506,6.59506,2019-10-31 20:32:48


In [57]:
imagesnew.columns = imagesnew.columns.str.strip()

In [58]:
images53 = imagesnew[imagesnew.locationId==53].reset_index(drop=True)
images53#.to_csv('join2.csv')

,locationId,imageId,imageKey,logtime
0,53,7a3abcc0-e3e0-11e9-90c3-45c3f76c0e47,53/7a3abcc0e3e011e990c345c3f76c0e47,2019-09-30 20:13:59
1,53,80de2660-e3e1-11e9-90c3-45c3f76c0e47,53/80de2660e3e111e990c345c3f76c0e47,2019-09-30 20:21:20
2,53,a96b7790-e3e1-11e9-90c3-45c3f76c0e47,53/a96b7790e3e111e990c345c3f76c0e47,2019-09-30 20:22:28
3,53,b8472d90-e3e1-11e9-8246-012e5e367d18,53/b8472d90e3e111e98246012e5e367d18,2019-09-30 20:22:53
4,53,ca2d3d60-e3e1-11e9-90c3-45c3f76c0e47,53/ca2d3d60e3e111e990c345c3f76c0e47,2019-09-30 20:23:23
...,...,...,...,...
8251,53,5160ff70-fc2d-11e9-8246-012e5e367d18,53/5160ff70fc2d11e98246012e5e367d18,2019-10-31 18:24:30
8252,53,9d412c20-fc2e-11e9-90c3-45c3f76c0e47,53/9d412c20fc2e11e990c345c3f76c0e47,2019-10-31 18:33:46
8253,53,ad91f5f0-fc2e-11e9-8246-012e5e367d18,53/ad91f5f0fc2e11e98246012e5e367d18,2019-10-31 18:34:14
8254,53,ba7c6340-fc2e-11e9-8246-012e5e367d18,53/ba7c6340fc2e11e98246012e5e367d18,2019-10-31 18:34:35


In [59]:
images53=images53[images53['logtime']>= '2019-10-1']
images53
                  

,locationId,imageId,imageKey,logtime
78,53,091a18f0-e42c-11e9-8246-012e5e367d18,53/091a18f0e42c11e98246012e5e367d18,2019-10-01 05:14:51
79,53,49ac3fb0-e42c-11e9-90c3-45c3f76c0e47,53/49ac3fb0e42c11e990c345c3f76c0e47,2019-10-01 05:16:39
80,53,5ffd23b0-e42c-11e9-90c3-45c3f76c0e47,53/5ffd23b0e42c11e990c345c3f76c0e47,2019-10-01 05:17:17
81,53,6ff7de40-e42c-11e9-8246-012e5e367d18,53/6ff7de40e42c11e98246012e5e367d18,2019-10-01 05:17:43
82,53,a0044880-e42c-11e9-8246-012e5e367d18,53/a0044880e42c11e98246012e5e367d18,2019-10-01 05:19:04
...,...,...,...,...
8251,53,5160ff70-fc2d-11e9-8246-012e5e367d18,53/5160ff70fc2d11e98246012e5e367d18,2019-10-31 18:24:30
8252,53,9d412c20-fc2e-11e9-90c3-45c3f76c0e47,53/9d412c20fc2e11e990c345c3f76c0e47,2019-10-31 18:33:46
8253,53,ad91f5f0-fc2e-11e9-8246-012e5e367d18,53/ad91f5f0fc2e11e98246012e5e367d18,2019-10-31 18:34:14
8254,53,ba7c6340-fc2e-11e9-8246-012e5e367d18,53/ba7c6340fc2e11e98246012e5e367d18,2019-10-31 18:34:35


In [60]:
df53.to_csv('try1.csv')

In [61]:
images53.to_csv('try2.csv')

In [62]:
images53

,locationId,imageId,imageKey,logtime
78,53,091a18f0-e42c-11e9-8246-012e5e367d18,53/091a18f0e42c11e98246012e5e367d18,2019-10-01 05:14:51
79,53,49ac3fb0-e42c-11e9-90c3-45c3f76c0e47,53/49ac3fb0e42c11e990c345c3f76c0e47,2019-10-01 05:16:39
80,53,5ffd23b0-e42c-11e9-90c3-45c3f76c0e47,53/5ffd23b0e42c11e990c345c3f76c0e47,2019-10-01 05:17:17
81,53,6ff7de40-e42c-11e9-8246-012e5e367d18,53/6ff7de40e42c11e98246012e5e367d18,2019-10-01 05:17:43
82,53,a0044880-e42c-11e9-8246-012e5e367d18,53/a0044880e42c11e98246012e5e367d18,2019-10-01 05:19:04
...,...,...,...,...
8251,53,5160ff70-fc2d-11e9-8246-012e5e367d18,53/5160ff70fc2d11e98246012e5e367d18,2019-10-31 18:24:30
8252,53,9d412c20-fc2e-11e9-90c3-45c3f76c0e47,53/9d412c20fc2e11e990c345c3f76c0e47,2019-10-31 18:33:46
8253,53,ad91f5f0-fc2e-11e9-8246-012e5e367d18,53/ad91f5f0fc2e11e98246012e5e367d18,2019-10-31 18:34:14
8254,53,ba7c6340-fc2e-11e9-8246-012e5e367d18,53/ba7c6340fc2e11e98246012e5e367d18,2019-10-31 18:34:35


In [63]:
tol = pd.Timedelta('10 second')
full = pd.merge_asof(df53, images53, left_on="logtime", right_on="logtime", direction="nearest", tolerance = tol)
full

,Date_Produced,Log_ID,Item_Name,Item_Type,PLU,Meal,Action_Taken,Action_Reason,Quantity,Pan_ID,...,Location_ID,Location,Served,Saved,Sold,Shrink,logtime,locationId,imageId,imageKey
0,2019-10-01,43100,Organic Rainbow Cherry Tomato,Menu Item,46000016104,Breakfast,Served,Start,3.88014,175,...,53,Westford,2.03014,0.0,2.03014,0.00000,2019-10-01 05:18:13,NaN,NaN,NaN
1,2019-10-01,43101,Organic Red Onions,Menu Item,46000055881,Breakfast,Served,Start,2.93215,175,...,53,Westford,1.08215,0.0,1.08215,0.00000,2019-10-01 05:19:07,53.0,a0044880-e42c-11e9-8246-012e5e367d18,53/a0044880e42c11e98246012e5e367d18
2,2019-10-01,43102,Red Cabbage,Menu Item,46000055765,Breakfast,Served,Start,2.46918,175,...,53,Westford,0.61918,0.0,0.61918,0.00000,2019-10-01 05:34:51,NaN,NaN,NaN
3,2019-10-01,43103,Paleo Friendly Roasted Red Bell Peppers,Menu Item,46000028993,Breakfast,Served,Start,2.91010,175,...,53,Westford,1.06010,0.0,1.06010,0.00000,2019-10-01 05:35:15,NaN,NaN,NaN
4,2019-10-01,43104,Cooked Red Beets,Menu Item,46000007337,Breakfast,Served,Start,4.05651,175,...,53,Westford,2.20651,0.0,2.20651,0.00000,2019-10-01 05:36:35,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7016,2019-10-31,69425,Pumpkin Tortellini,Menu Item,46000029503,Breakfast,Discarded,Over Produced,4.27697,173,...,53,Westford,0.00000,0.0,-2.73697,2.73697,2019-10-31 20:31:39,NaN,NaN,NaN
7017,2019-10-31,69427,Chicken Cacciatore,Menu Item,46000005678,Breakfast,Discarded,Over Produced,6.08476,173,...,53,Westford,0.00000,0.0,-4.54476,4.54476,2019-10-31 20:32:04,NaN,NaN,NaN
7018,2019-10-31,69428,Garlic Parm Pork Loin with Broccoli Rabe,Menu Item,46000029505,Breakfast,Discarded,Over Produced,2.29281,173,...,53,Westford,0.00000,0.0,-0.75281,0.75281,2019-10-31 20:32:21,NaN,NaN,NaN
7019,2019-10-31,69430,Macaroni & Cheese,Menu Item,46000060414,Breakfast,Discarded,Over Produced,8.13506,173,...,53,Westford,0.00000,0.0,-6.59506,6.59506,2019-10-31 20:32:48,NaN,NaN,NaN


In [64]:
full.to_csv('oct53.csv')